In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 46 kB/s 
     |████████████████████████████████| 199 kB 34.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=56c71b6e5db485ab610c0ff818c327df840c23164a4fa9f19c2be0c71c185e12
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 htt

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
downloaded = drive.CreateFile({'id':"1uOMdWNsxIJ3WHgvde726e169idKgs--8"})
downloaded.GetContentFile('big-mac-source-data2022.csv') 

In [5]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [6]:
sqlContext = SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [7]:
df = sqlContext.read.csv('big-mac-source-data2022.csv', header=True, inferSchema=True)

In [8]:
df.rdd

MapPartitionsRDD[14] at javaToPython at NativeMethodAccessorImpl.java:0

In [9]:
df.schema

StructType([StructField('name', StringType(), True), StructField('iso_a3', StringType(), True), StructField('currency_code', StringType(), True), StructField('local_price', DoubleType(), True), StructField('dollar_ex', DoubleType(), True), StructField('GDP_dollar', DoubleType(), True), StructField('date', TimestampType(), True)])

In [10]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- iso_a3: string (nullable = true)
 |-- currency_code: string (nullable = true)
 |-- local_price: double (nullable = true)
 |-- dollar_ex: double (nullable = true)
 |-- GDP_dollar: double (nullable = true)
 |-- date: timestamp (nullable = true)



In [11]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType

In [12]:
Schema = StructType([ StructField("name", StringType(), True),
                      StructField("iso_a3", StringType(), True),
                      StructField("currency_code", StringType(), True),
                      StructField("local_price", DoubleType(), True),
                      StructField("dollar_ex", DoubleType(), True),
                      StructField("GDP_dollar", DoubleType(), True),
                     StructField("date", TimestampType(), True)
                    ])

In [13]:
df = sqlContext.read.csv('big-mac-source-data2022.csv', header=True, schema=Schema)

In [14]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- iso_a3: string (nullable = true)
 |-- currency_code: string (nullable = true)
 |-- local_price: double (nullable = true)
 |-- dollar_ex: double (nullable = true)
 |-- GDP_dollar: double (nullable = true)
 |-- date: timestamp (nullable = true)



In [15]:
df.count()

1876

In [16]:
df.head(5)

[Row(name='Argentina', iso_a3='ARG', currency_code='ARS', local_price=2.5, dollar_ex=1.0, GDP_dollar=None, date=datetime.datetime(2000, 4, 1, 0, 0)),
 Row(name='Australia', iso_a3='AUS', currency_code='AUD', local_price=2.59, dollar_ex=1.68, GDP_dollar=None, date=datetime.datetime(2000, 4, 1, 0, 0)),
 Row(name='Brazil', iso_a3='BRA', currency_code='BRL', local_price=2.95, dollar_ex=1.79, GDP_dollar=None, date=datetime.datetime(2000, 4, 1, 0, 0)),
 Row(name='Britain', iso_a3='GBR', currency_code='GBP', local_price=1.9, dollar_ex=0.632911392, GDP_dollar=None, date=datetime.datetime(2000, 4, 1, 0, 0)),
 Row(name='Canada', iso_a3='CAN', currency_code='CAD', local_price=2.85, dollar_ex=1.47, GDP_dollar=None, date=datetime.datetime(2000, 4, 1, 0, 0))]

In [17]:
df.show()

+--------------+------+-------------+-----------+-----------+----------+-------------------+
|          name|iso_a3|currency_code|local_price|  dollar_ex|GDP_dollar|               date|
+--------------+------+-------------+-----------+-----------+----------+-------------------+
|     Argentina|   ARG|          ARS|        2.5|        1.0|      null|2000-04-01 00:00:00|
|     Australia|   AUS|          AUD|       2.59|       1.68|      null|2000-04-01 00:00:00|
|        Brazil|   BRA|          BRL|       2.95|       1.79|      null|2000-04-01 00:00:00|
|       Britain|   GBR|          GBP|        1.9|0.632911392|      null|2000-04-01 00:00:00|
|        Canada|   CAN|          CAD|       2.85|       1.47|      null|2000-04-01 00:00:00|
|         Chile|   CHL|          CLP|     1260.0|      514.0|      null|2000-04-01 00:00:00|
|         China|   CHN|          CNY|        9.9|       8.28|      null|2000-04-01 00:00:00|
|Czech Republic|   CZE|          CZK|      54.37|       39.1|      nul

In [18]:
df.describe().show()

+-------+---------+------+-------------+------------------+------------------+------------------+
|summary|     name|iso_a3|currency_code|       local_price|         dollar_ex|        GDP_dollar|
+-------+---------+------+-------------+------------------+------------------+------------------+
|  count|     1876|  1876|         1876|              1876|              1876|              1263|
|   mean|     null|  null|         null|16264.709962031364| 4859.870233113078|26950.650459675293|
| stddev|     null|  null|         null|401270.68922267726|102478.19239009761|21210.691703476616|
|    min|Argentina|   ARE|          AED|               0.0|               0.0|          1049.749|
|    max|  Vietnam|   ZAF|          ZAR|           1.602E7|       3613989.071|        100578.968|
+-------+---------+------+-------------+------------------+------------------+------------------+



In [19]:
df.summary().show()

+-------+---------+------+-------------+------------------+------------------+------------------+
|summary|     name|iso_a3|currency_code|       local_price|         dollar_ex|        GDP_dollar|
+-------+---------+------+-------------+------------------+------------------+------------------+
|  count|     1876|  1876|         1876|              1876|              1876|              1263|
|   mean|     null|  null|         null|16264.709962031364| 4859.870233113078|26950.650459675293|
| stddev|     null|  null|         null|401270.68922267726|102478.19239009761|21210.691703476616|
|    min|Argentina|   ARE|          AED|               0.0|               0.0|          1049.749|
|    25%|     null|  null|         null|              4.45|             1.026|          9054.914|
|    50%|     null|  null|         null|             14.99|            5.5014|         20812.586|
|    75%|     null|  null|         null|              87.0|            32.875|         43527.994|
|    max|  Vietnam| 

In [20]:
df.columns

['name',
 'iso_a3',
 'currency_code',
 'local_price',
 'dollar_ex',
 'GDP_dollar',
 'date']

In [21]:
df.groupby('name').agg({'local_price': 'max', 'dollar_ex': 'max'}).show()

+-----------+-----------------+----------------+
|       name|   max(dollar_ex)|max(local_price)|
+-----------+-----------------+----------------+
|     Russia|          77.4175|           137.0|
|     Sweden|             10.4|            54.0|
|Philippines|             55.9|           163.0|
|   Malaysia|           4.4715|            9.99|
|  Singapore|             1.82|             5.9|
|    Britain|       0.82877507|            3.59|
|     Turkey|        1600500.0|       4000000.0|
|    Germany|      0.955246692|            4.46|
|     Jordan|          0.71005|             2.3|
|     France|      0.955246692|            4.35|
|     Greece|      0.955246692|             3.4|
|  Sri Lanka|           202.41|           840.0|
|     Taiwan|             34.8|            79.0|
|   Slovakia|0.899159286067527|             3.8|
|  Argentina|         105.0165|           450.0|
|    Belgium|      0.955246692|            4.55|
|      Qatar|          3.64175|            13.0|
|    Finland|      0

In [23]:
df.groupby('name').mean('local_price').show()

+-----------+------------------+
|       name|  avg(local_price)|
+-----------+------------------+
|     Russia| 87.54305555555555|
|     Sweden| 41.46478670138889|
|Philippines|116.28914285714285|
|   Malaysia| 7.297222222222222|
|  Singapore|4.6447222222222235|
|    Britain| 2.642222222222222|
|     Turkey| 344126.3305882353|
|    Germany| 3.898181818181818|
|     Jordan|           2.16875|
|     France|4.0159090909090915|
|     Greece|3.2133181818181815|
|  Sri Lanka|406.93548387096774|
|     Taiwan| 73.41666666666667|
|   Slovakia|              3.45|
|  Argentina| 67.04055555555556|
|    Belgium| 4.013636363636364|
|      Qatar|             12.75|
|    Finland| 4.376363636363637|
|  Nicaragua|            117.75|
|       Peru|10.090666666666662|
+-----------+------------------+
only showing top 20 rows



In [24]:
df.groupby('name').max('local_price').show()

+-----------+----------------+
|       name|max(local_price)|
+-----------+----------------+
|     Russia|           137.0|
|     Sweden|            54.0|
|Philippines|           163.0|
|   Malaysia|            9.99|
|  Singapore|             5.9|
|    Britain|            3.59|
|     Turkey|       4000000.0|
|    Germany|            4.46|
|     Jordan|             2.3|
|     France|            4.35|
|     Greece|             3.4|
|  Sri Lanka|           840.0|
|     Taiwan|            79.0|
|   Slovakia|             3.8|
|  Argentina|           450.0|
|    Belgium|            4.55|
|      Qatar|            13.0|
|    Finland|            4.95|
|  Nicaragua|           134.0|
|       Peru|            12.9|
+-----------+----------------+
only showing top 20 rows



In [25]:
df.groupby('name').min('local_price').show()

+-----------+----------------+
|       name|min(local_price)|
+-----------+----------------+
|     Russia|            35.0|
|     Sweden|            24.0|
|Philippines|            59.0|
|   Malaysia|            4.52|
|  Singapore|             3.2|
|    Britain|            1.88|
|     Turkey|             4.0|
|    Germany|             3.4|
|     Jordan|            1.95|
|     France|             3.5|
|     Greece|             2.6|
|  Sri Lanka|           140.0|
|     Taiwan|            69.0|
|   Slovakia|             3.0|
|  Argentina|             2.5|
|    Belgium|             3.7|
|      Qatar|            12.0|
|    Finland|            3.75|
|  Nicaragua|           103.0|
|       Peru|             7.9|
+-----------+----------------+
only showing top 20 rows



In [26]:
from pyspark.sql.functions import count, avg, min, max, round

In [27]:
df.groupby('name').agg(avg('local_price'), min('local_price'), max('local_price')).show()

+-----------+------------------+----------------+----------------+
|       name|  avg(local_price)|min(local_price)|max(local_price)|
+-----------+------------------+----------------+----------------+
|     Russia| 87.54305555555555|            35.0|           137.0|
|     Sweden| 41.46478670138889|            24.0|            54.0|
|Philippines|116.28914285714285|            59.0|           163.0|
|   Malaysia| 7.297222222222222|            4.52|            9.99|
|  Singapore|4.6447222222222235|             3.2|             5.9|
|    Britain| 2.642222222222222|            1.88|            3.59|
|     Turkey| 344126.3305882353|             4.0|       4000000.0|
|    Germany| 3.898181818181818|             3.4|            4.46|
|     Jordan|           2.16875|            1.95|             2.3|
|     France|4.0159090909090915|             3.5|            4.35|
|     Greece|3.2133181818181815|             2.6|             3.4|
|  Sri Lanka|406.93548387096774|           140.0|           84

In [28]:
from pyspark.sql.functions import pandas_udf, PandasUDFType

@pandas_udf('double', PandasUDFType.GROUPED_AGG)
def max_udf(v):
    return v.max()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/pandas/functions.py:398: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  UserWarning,


In [29]:
df.groupby('name').agg(max_udf(df.local_price)).show()

+--------------+--------------------+
|          name|max_udf(local_price)|
+--------------+--------------------+
|     Argentina|               450.0|
|     Australia|                6.55|
|       Austria|                3.92|
|    Azerbaijan|                 4.5|
|       Bahrain|                 1.5|
|       Belgium|                4.55|
|        Brazil|                22.9|
|       Britain|                3.59|
|        Canada|                6.88|
|         Chile|              3100.0|
|         China|                24.4|
|      Colombia|             12950.0|
|    Costa Rica|              2650.0|
|       Croatia|                24.0|
|Czech Republic|                89.0|
|       Denmark|               34.59|
|         Egypt|                42.5|
|       Estonia|                 3.4|
|     Euro area|                4.42|
|       Finland|                4.95|
+--------------+--------------------+
only showing top 20 rows



In [31]:
info_ar = df.filter('name = "Argentina"').drop('name', 'iso_a3', 'currency_code', 'GDP_dollar')
info_ar.show()

+-----------+---------+-------------------+
|local_price|dollar_ex|               date|
+-----------+---------+-------------------+
|        2.5|      1.0|2000-04-01 00:00:00|
|        2.5|      1.0|2001-04-01 00:00:00|
|        2.5|     3.13|2002-04-01 00:00:00|
|        4.1|     2.88|2003-04-01 00:00:00|
|       4.36|     2.95|2004-05-01 00:00:00|
|       4.75|    2.897|2005-06-01 00:00:00|
|       4.75|   3.0638|2006-01-01 00:00:00|
|        7.0|   3.0565|2006-05-01 00:00:00|
|       8.25|  3.08875|2007-01-01 00:00:00|
|       8.25|   3.0915|2007-06-01 00:00:00|
|       11.0|   3.0195|2008-06-01 00:00:00|
|       11.5|  3.81125|2009-07-01 00:00:00|
|        7.0|   3.7985|2010-01-01 00:00:00|
|       14.0|  3.93375|2010-07-01 00:00:00|
|       20.0|   4.1325|2011-07-01 00:00:00|
|       20.0|   4.3135|2012-01-01 00:00:00|
|       19.0|  4.56625|2012-07-01 00:00:00|
|       19.0|   4.9765|2013-01-01 00:00:00|
|       21.0|   5.4125|2013-07-01 00:00:00|
|       21.0|  6.92125|2014-01-0

In [32]:
import time
import datetime

In [33]:
data_ar = info_ar.rdd\
                 .map(lambda x: (time.mktime(x.date.timetuple()), x.local_price))\
                 .collect()

In [34]:
data_ar_list = list()
for data in data_ar:
    data_list = list(data)
    item = list()
    item.append(data_list[0]*1000)
    item.append(float(data_list[1]))
    data_ar_list.append(item)

In [35]:
data_ar_usd = df.filter('name = "Argentina"')\
                .drop('name', 'iso_a3', 'currency_code', 'GDP_dollar')\
                .rdd\
                .map(lambda x: (int(time.mktime(x.date.timetuple())), float(x.local_price)/float(x.dollar_ex)))\
                .collect()

In [36]:
data_ar_usd

[(954547200, 2.5),
 (986083200, 2.5),
 (1017619200, 0.7987220447284346),
 (1049155200, 1.423611111111111),
 (1083369600, 1.4779661016949153),
 (1117584000, 1.6396272005522956),
 (1136073600, 1.550362295188981),
 (1146441600, 2.290201210534925),
 (1167609600, 2.670983407527317),
 (1180656000, 2.668607472100922),
 (1212278400, 3.642987249544627),
 (1246406400, 3.0173827484421123),
 (1262304000, 1.842832697117283),
 (1277942400, 3.558945027009851),
 (1309478400, 4.839685420447671),
 (1325376000, 4.636606004404776),
 (1341100800, 4.160963591568573),
 (1356998400, 3.8179443383904355),
 (1372636800, 3.8799076212471135),
 (1388534400, 3.0341340075853354),
 (1404172800, 2.5707727620504977),
 (1420070400, 3.2520325203252036),
 (1435708800, 3.0651340996168583),
 (1451606400, 2.3897025544472004),
 (1467331200, 3.3478406427854033),
 (1483228800, 3.4683903515686585),
 (1498867200, 4.1255341093266535),
 (1514764800, 3.9603960396039604),
 (1530403200, 2.7051397655545535),
 (1546300800, 2.002402883460

In [37]:
df.groupBy().agg({ 'local_price': 'mean' }).show()

+------------------+
|  avg(local_price)|
+------------------+
|16264.709962031364|
+------------------+



In [38]:
df.fillna(0).show()

+--------------+------+-------------+-----------+-----------+----------+-------------------+
|          name|iso_a3|currency_code|local_price|  dollar_ex|GDP_dollar|               date|
+--------------+------+-------------+-----------+-----------+----------+-------------------+
|     Argentina|   ARG|          ARS|        2.5|        1.0|       0.0|2000-04-01 00:00:00|
|     Australia|   AUS|          AUD|       2.59|       1.68|       0.0|2000-04-01 00:00:00|
|        Brazil|   BRA|          BRL|       2.95|       1.79|       0.0|2000-04-01 00:00:00|
|       Britain|   GBR|          GBP|        1.9|0.632911392|       0.0|2000-04-01 00:00:00|
|        Canada|   CAN|          CAD|       2.85|       1.47|       0.0|2000-04-01 00:00:00|
|         Chile|   CHL|          CLP|     1260.0|      514.0|       0.0|2000-04-01 00:00:00|
|         China|   CHN|          CNY|        9.9|       8.28|       0.0|2000-04-01 00:00:00|
|Czech Republic|   CZE|          CZK|      54.37|       39.1|       0.

In [39]:
df.replace({'Argentina': 'AR'}, subset=['name']).show()

+--------------+------+-------------+-----------+-----------+----------+-------------------+
|          name|iso_a3|currency_code|local_price|  dollar_ex|GDP_dollar|               date|
+--------------+------+-------------+-----------+-----------+----------+-------------------+
|            AR|   ARG|          ARS|        2.5|        1.0|      null|2000-04-01 00:00:00|
|     Australia|   AUS|          AUD|       2.59|       1.68|      null|2000-04-01 00:00:00|
|        Brazil|   BRA|          BRL|       2.95|       1.79|      null|2000-04-01 00:00:00|
|       Britain|   GBR|          GBP|        1.9|0.632911392|      null|2000-04-01 00:00:00|
|        Canada|   CAN|          CAD|       2.85|       1.47|      null|2000-04-01 00:00:00|
|         Chile|   CHL|          CLP|     1260.0|      514.0|      null|2000-04-01 00:00:00|
|         China|   CHN|          CNY|        9.9|       8.28|      null|2000-04-01 00:00:00|
|Czech Republic|   CZE|          CZK|      54.37|       39.1|      nul

In [40]:
df.selectExpr("*","local_price * dollar_ex as usd_price").show()

+--------------+------+-------------+-----------+-----------+----------+-------------------+------------------+
|          name|iso_a3|currency_code|local_price|  dollar_ex|GDP_dollar|               date|         usd_price|
+--------------+------+-------------+-----------+-----------+----------+-------------------+------------------+
|     Argentina|   ARG|          ARS|        2.5|        1.0|      null|2000-04-01 00:00:00|               2.5|
|     Australia|   AUS|          AUD|       2.59|       1.68|      null|2000-04-01 00:00:00|4.3511999999999995|
|        Brazil|   BRA|          BRL|       2.95|       1.79|      null|2000-04-01 00:00:00| 5.280500000000001|
|       Britain|   GBR|          GBP|        1.9|0.632911392|      null|2000-04-01 00:00:00|      1.2025316448|
|        Canada|   CAN|          CAD|       2.85|       1.47|      null|2000-04-01 00:00:00|            4.1895|
|         Chile|   CHL|          CLP|     1260.0|      514.0|      null|2000-04-01 00:00:00|          64

In [41]:
df.groupBy("name").pivot("date").min('local_price').show()

+-----------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|       name|2000-04-01 00:00:00|2001-04-01 00:00:00|2002-04-01 00:00:00|2003-04-01 00:00:00|2004-05-01 00:00:00|2005-06-01 00:00:00|2006-01-01 00:00:00|2006-05-01 00:00:00|2007-01-01 00:00:00|2007-06-01 00:00:00|2008-06-01 00:00:00|2009-07-01 00:00:00|2010-01-01 00

In [42]:
df.registerTempTable('bigmac')

/usr/local/lib/python3.7/dist-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [43]:
sqlContext.sql('select name, max(local_price) from bigmac group by name order by max(local_price) asc limit 10 ').show()

+--------+----------------+
|    name|max(local_price)|
+--------+----------------+
|    Oman|            1.21|
|  Kuwait|            1.25|
| Bahrain|             1.5|
|  Jordan|             2.3|
|Slovenia|             2.8|
|  Latvia|            2.85|
|  Greece|             3.4|
| Estonia|             3.4|
| Britain|            3.59|
|Slovakia|             3.8|
+--------+----------------+

